In [94]:
from keras.models import Sequential , Model
from keras.layers import Dense ,  BatchNormalization , Reshape , Input , Flatten
from keras.layers import Conv2D , MaxPool2D , Conv2DTranspose , UpSampling2D , ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Activation
from keras.layers import Dropout

from keras.initializers import truncated_normal , constant , random_normal

from keras.optimizers import Adam

from keras.datasets import mnist

In [95]:
import os

import matplotlib as plt
import numpy as np

import gc

import keras.backend as K

%matplotlib inline

In [96]:
WIDTH = 96
HEIGHT = 96
CHANNEL = 3

LATENT_DIM = 100 #latent variable z sample from normal distribution

BATCH_SIZE = 64
EPOCHS = 10

PATH = 'faces/'

#生成多少个图像 长*宽
ROW = 5
COL = 5


In [97]:

load_index = 0

images_name = os.listdir(PATH)

IMAGES_COUNT = len(images_name)


In [98]:
'''
(X_train , y_train),(X_test , y_test) = mnist.load_data()
X_train = X_train/127.5-1
X_train = np.expand_dims(X_train , 3)
'''

'\n(X_train , y_train),(X_test , y_test) = mnist.load_data()\nX_train = X_train/127.5-1\nX_train = np.expand_dims(X_train , 3)\n'

In [99]:
'''
def load_mnist():
    return X_train[np.random.randint(0, X_train.shape[0], BATCH_SIZE)]
    
def write_image_mnist(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = generated_image*0.5+0.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('mnist_dcgan/No.%d.png' % epoch)
    plt.pyplot.close()

'''

"\ndef load_mnist():\n    return X_train[np.random.randint(0, X_train.shape[0], BATCH_SIZE)]\n    \ndef write_image_mnist(epoch):\n    \n    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))\n    generated_image = generator_i.predict(noise)\n    generated_image = generated_image*0.5+0.5\n    \n    fig , axes = plt.pyplot.subplots(ROW , COL)\n    \n    count=0\n    \n    for i in range(ROW):\n        for j in range(COL):\n            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')\n            axes[i][j].axis('off')\n            count += 1\n            \n    fig.savefig('mnist_dcgan/No.%d.png' % epoch)\n    plt.pyplot.close()\n\n"

In [100]:

def load_image(batch_size = BATCH_SIZE):
    global load_index
    
    images = []
    
    for i in range(batch_size):
        images.append(plt.image.imread(PATH + images_name[(load_index + i) % IMAGES_COUNT]))
    
    load_index += batch_size
    
    return (np.array(images)/127.5)-1.0

def write_image(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = np.floor(((generated_image+1.0)*127.5))
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count])
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('generated_faces_dcgan/No.%d.png' % epoch)
    plt.pyplot.close()
    
    #plt.image.imsave('images/'+str(epoch)+'.jpg')


In [101]:
def conv2d(output_size):
    return Conv2D(output_size , kernel_size=(5,5) , strides=(2,2) , padding='same' , kernel_initializer=truncated_normal(stddev=0.02) , bias_initializer=constant(0.0))

def dense(output_size):
    return Dense(output_size , kernel_initializer=random_normal(stddev=0.02) , bias_initializer=constant(0.0))

def deconv2d(output_size):
    return Conv2DTranspose(output_size , kernel_size=(5,5) , strides=(2,2) , padding='same' , kernel_initializer=random_normal(stddev=0.02) , bias_initializer=constant(0.0))

def batch_norm():
    return BatchNormalization(momentum=0.9 , epsilon=1e-5)


In [102]:
def generator():
    #sample from noise z
    model = Sequential(name='generator')
    
    #cartoon 图像使用 96*96*3
    model.add(Dense(6*6*8*64 , input_shape=(LATENT_DIM,) , kernel_initializer=random_normal(stddev=0.02) , bias_initializer=constant(0.0)))
    
    model.add(Reshape((6, 6, 64*8)))
    
    model.add(batch_norm())
    model.add(Activation('relu'))

    model.add(deconv2d(64*4))
    model.add(batch_norm())
    model.add(Activation('relu'))
    
    model.add(deconv2d(64*2))
    model.add(batch_norm())
    model.add(Activation('relu'))
    
    model.add(deconv2d(64*1))
    model.add(batch_norm())
    model.add(Activation('relu'))
    
    model.add(deconv2d(3))
    model.add(Activation('tanh'))
    
    #model.summary()
    
    noise = Input(shape=(LATENT_DIM , ) , name='input1')
    image = model(noise)
    
    return Model(noise , image , name='generator_Model')

In [103]:
def discriminator():
    #input a image to discriminate real or fake
    model = Sequential(name='discriminator')
    
    model.add(Conv2D(filters=64 , kernel_size=(5,5) , strides=(2,2) , padding='same' , input_shape=(WIDTH , HEIGHT , CHANNEL) , kernel_initializer=truncated_normal(stddev=0.02) , bias_initializer=constant(0.0) , name='conv1'))
    model.add(LeakyReLU(0.2))
    #model.add(Dropout(0.25))
    
    #model.add(BatchNormalization(momentum=0.8))
    model.add(conv2d(64*2))
    model.add(batch_norm())
    model.add(LeakyReLU(0.2))
    
    model.add(conv2d(64*4))
    model.add(batch_norm())  
    model.add(LeakyReLU(0.2))

    
    model.add(conv2d(64*8))
    model.add(batch_norm())  
    model.add(LeakyReLU(0.2))

    model.add(Flatten())
    
    #===
    model.add(dense(1024))
    model.add(LeakyReLU(0.2))
    
    model.add(dense(256))
    model.add(LeakyReLU(0.2))
    #===
    model.add(dense(1))
    model.add(Activation('sigmoid'))
    
    #model.summary()
    
    image = Input(shape=(WIDTH , HEIGHT , CHANNEL) , name='input1')
    validity = model(image)
    
    return Model(image , validity , name='discriminator_Model')

In [104]:
def combined_model(generator_i , discriminator_i):
    #生成器和判别器组合成整体
    z = Input(shape=(LATENT_DIM , ) , name='z')
    
    image = generator_i(z)
    
    #print(image.shape)
    
    discriminator_i.trainable = False
    validity = discriminator_i(image)
    
    return Model(z , validity , name='combined_model')

In [105]:
adam = Adam(lr = 0.0002 , beta_1=0.5)

In [106]:
discriminator_i = discriminator()
discriminator_i.compile(optimizer=adam , loss='binary_crossentropy' , metrics=['accuracy'])

generator_i = generator()

combined_model_i = combined_model(generator_i , discriminator_i)


combined_model_i.compile(optimizer=adam , loss='binary_crossentropy')

In [ ]:
real_labels = np.ones(shape=(BATCH_SIZE , 1)) #真实样本label为1
fake_labels = np.zeros(shape=(BATCH_SIZE , 1)) #假样本label为0

for i in range(1000):
    for j in range(int(IMAGES_COUNT/BATCH_SIZE)):
        noise = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))

        real_image = load_image()
        #real_image = load_mnist()
        #训练判别器
        fake_image = generator_i.predict(noise)

        real_loss = discriminator_i.train_on_batch(real_image , real_labels)
        fake_loss = discriminator_i.train_on_batch(fake_image , fake_labels)

        loss = np.add(real_loss , fake_loss)/2

        #训练生成器
        noise2 = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
        generator_loss = combined_model_i.train_on_batch(noise2 , real_labels)

        print('epoch:%d batch:%d loss:%f accu:%f gene_loss:%f' % (i , j , loss[0] , loss[1] , generator_loss))

    write_image(i)
    #write_image_mnist(i)
    
write_image(999)
#write_image_mnist(999)


C:\Anaconda3\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 batch:0 loss:1.532499 accu:0.031250 gene_loss:0.709593
epoch:0 batch:1 loss:0.472405 accu:0.523438 gene_loss:1.578565
epoch:0 batch:2 loss:0.371663 accu:1.000000 gene_loss:2.661038
epoch:0 batch:3 loss:0.224815 accu:0.992188 gene_loss:4.418355
epoch:0 batch:4 loss:0.122419 accu:0.992188 gene_loss:4.013967
epoch:0 batch:5 loss:0.157530 accu:1.000000 gene_loss:8.017440
epoch:0 batch:6 loss:0.093054 accu:0.968750 gene_loss:6.039183
epoch:0 batch:7 loss:0.162589 accu:0.984375 gene_loss:10.644134
epoch:0 batch:8 loss:0.067714 accu:0.976562 gene_loss:11.256130
epoch:0 batch:9 loss:0.026691 accu:1.000000 gene_loss:4.857416
epoch:0 batch:10 loss:0.458809 accu:0.617188 gene_loss:15.747258
epoch:0 batch:11 loss:0.324746 accu:0.835938 gene_loss:14.454495
epoch:0 batch:12 loss:0.003791 accu:1.000000 gene_loss:8.059889
epoch:0 batch:13 loss:1.871708 accu:0.500000 gene_loss:13.974639
epoch:0 batch:14 loss:0.496819 accu:0.781250 gene_loss:10.880688
epoch:0 batch:15 loss:0.076631 accu:0.976562

epoch:0 batch:128 loss:0.304907 accu:0.882812 gene_loss:5.646421
epoch:0 batch:129 loss:0.150591 accu:0.953125 gene_loss:3.472129
epoch:0 batch:130 loss:0.600873 accu:0.640625 gene_loss:6.791640
epoch:0 batch:131 loss:0.494658 accu:0.742188 gene_loss:3.820701
epoch:0 batch:132 loss:0.215073 accu:0.898438 gene_loss:2.978500
epoch:0 batch:133 loss:0.092377 accu:0.960938 gene_loss:1.606507
epoch:0 batch:134 loss:0.085242 accu:0.976562 gene_loss:0.337958
epoch:0 batch:135 loss:0.050696 accu:1.000000 gene_loss:0.363656
epoch:0 batch:136 loss:0.023948 accu:0.992188 gene_loss:0.272325
epoch:0 batch:137 loss:0.024259 accu:1.000000 gene_loss:0.107272
epoch:0 batch:138 loss:0.020259 accu:1.000000 gene_loss:0.071245
epoch:0 batch:139 loss:0.030349 accu:1.000000 gene_loss:0.070611
epoch:0 batch:140 loss:0.014010 accu:1.000000 gene_loss:0.111581
epoch:0 batch:141 loss:0.011714 accu:1.000000 gene_loss:0.074572
epoch:0 batch:142 loss:0.009197 accu:1.000000 gene_loss:0.080034
epoch:0 batch:143 loss:0.

epoch:0 batch:255 loss:0.048937 accu:0.984375 gene_loss:0.497876
epoch:0 batch:256 loss:0.054456 accu:1.000000 gene_loss:0.026002
epoch:0 batch:257 loss:0.031698 accu:1.000000 gene_loss:0.012093
epoch:0 batch:258 loss:0.035077 accu:1.000000 gene_loss:0.028514
epoch:0 batch:259 loss:0.097321 accu:0.976562 gene_loss:1.651345
epoch:0 batch:260 loss:0.127065 accu:0.945312 gene_loss:0.203785
epoch:0 batch:261 loss:0.322949 accu:0.906250 gene_loss:6.841724
epoch:0 batch:262 loss:1.593414 accu:0.453125 gene_loss:3.675822
epoch:0 batch:263 loss:0.199595 accu:0.929688 gene_loss:2.614352
epoch:0 batch:264 loss:0.392566 accu:0.835938 gene_loss:1.682814
epoch:0 batch:265 loss:0.319199 accu:0.882812 gene_loss:2.032523
epoch:0 batch:266 loss:0.335020 accu:0.875000 gene_loss:1.940268
epoch:0 batch:267 loss:0.550613 accu:0.734375 gene_loss:2.644644
epoch:0 batch:268 loss:0.343235 accu:0.828125 gene_loss:0.860222
epoch:0 batch:269 loss:0.158120 accu:0.945312 gene_loss:0.965958
epoch:0 batch:270 loss:0.

epoch:0 batch:382 loss:0.172445 accu:0.945312 gene_loss:2.111742
epoch:0 batch:383 loss:0.259708 accu:0.851562 gene_loss:0.592207
epoch:0 batch:384 loss:0.377197 accu:0.796875 gene_loss:4.721841
epoch:0 batch:385 loss:0.737468 accu:0.617188 gene_loss:0.351244
epoch:0 batch:386 loss:0.390227 accu:0.851562 gene_loss:1.421335
epoch:0 batch:387 loss:0.126522 accu:0.953125 gene_loss:1.291909
epoch:0 batch:388 loss:0.792786 accu:0.585938 gene_loss:3.720171
epoch:0 batch:389 loss:0.693512 accu:0.687500 gene_loss:2.534351
epoch:0 batch:390 loss:0.624794 accu:0.695312 gene_loss:1.710853
epoch:0 batch:391 loss:0.236309 accu:0.953125 gene_loss:2.003317
epoch:0 batch:392 loss:0.220816 accu:0.953125 gene_loss:2.247858
epoch:0 batch:393 loss:0.186455 accu:0.953125 gene_loss:3.116645
epoch:0 batch:394 loss:0.294249 accu:0.875000 gene_loss:2.635972
epoch:0 batch:395 loss:0.161502 accu:0.968750 gene_loss:2.382519
epoch:0 batch:396 loss:0.222917 accu:0.953125 gene_loss:1.594467
epoch:0 batch:397 loss:0.

epoch:0 batch:509 loss:0.729638 accu:0.578125 gene_loss:1.845765
epoch:0 batch:510 loss:0.729393 accu:0.578125 gene_loss:2.595599


In [70]:
real_labels.shape

(64, 1)

In [53]:
discriminator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
discriminator (Sequential)   (None, 1)                 533505    
Total params: 533,505
Trainable params: 0
Non-trainable params: 533,505
_________________________________________________________________


In [54]:
generator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 100)               0         
_________________________________________________________________
generator (Sequential)       (None, 28, 28, 1)         1097744   
Total params: 1,097,744
Trainable params: 1,095,184
Non-trainable params: 2,560
_________________________________________________________________


In [39]:
combined_model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               (None, 100)               0         
_________________________________________________________________
generator_Model (Model)      (None, 96, 96, 3)         29029120  
_________________________________________________________________
discriminator_Model (Model)  (None, 1)                 14320641  
Total params: 43,349,761
Trainable params: 29,025,536
Non-trainable params: 14,324,225
_________________________________________________________________


In [ ]:
gc.collect()

In [29]:
modeli = Sequential(name='generator')

#cartioon 图像使用 96*96*3
modeli.add(Dense(6*6*8*64 , input_shape=(LATENT_DIM,) , kernel_initializer=random_normal(stddev=0.02) , bias_initializer=constant(0.0)))

modeli.add(Reshape((6, 6, 64*8)))
modeli.add(batch_norm())
modeli.add(LeakyReLU(0.2))

modeli.add(deconv2d(64*4))
modeli.add(batch_norm())
modeli.add(LeakyReLU(0.2))

modeli.add(deconv2d(64*2))
modeli.add(batch_norm())
modeli.add(LeakyReLU(0.2))

modeli.add(deconv2d(64*1))
modeli.add(batch_norm())
modeli.add(LeakyReLU(0.2))

modeli.add(deconv2d(3))
modeli.add(Activation('tanh'))

In [41]:
32*400

12800

In [ ]:
gc.collect()

In [18]:
gc.collect()

24

In [15]:
load_index

7872

In [88]:
IMAGES_COUNT/BATCH_SIZE

800.359375

In [89]:
800*64

51200

In [90]:
IMAGES_COUNT

51223

In [ ]:
22:9:30